In [5]:
# =============================================================================
# Importation des bibliothèques nécessaires
# =============================================================================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import heapq

# =============================================================================
# Partie 1 : Configuration de l'environnement
# =============================================================================
GRID_SIZE = 6
START_POS = (0, 0)  # Format (ligne, colonne)
GOAL_POS = (5, 5)
OBSTACLE_POS = (3, 3)

# =============================================================================
# Partie 2 : Implémentation de l'agent "stupide" (Réactif)
# =============================================================================
# Règle : Avancer toujours à droite. S'arrêter si le prochain pas est un obstacle
# ou si on atteint le bord de la grille.

path_reflex = [START_POS]
current_pos_reflex = list(START_POS)

for _ in range(GRID_SIZE - 1):
    # Prochaine position potentielle
    next_pos = (current_pos_reflex[0], current_pos_reflex[1] + 1)
    
    # Arrêter si le prochain pas est l'obstacle
    if next_pos == OBSTACLE_POS:
        break  # L'agent est bloqué et s'arrête
        
    # Mettre à jour la position et l'ajouter au chemin
    current_pos_reflex[1] += 1
    path_reflex.append(tuple(current_pos_reflex))

# =============================================================================
# Partie 3 : Implémentation de l'agent intelligent (Basé sur un but)
# =============================================================================
# Utilise l'algorithme A* pour trouver le chemin le plus court en évitant l'obstacle.

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid_size, start, goal, obstacle):
    neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}
    oheap = []
    heapq.heappush(oheap, (fscore[start], start))
    while oheap:
        current = heapq.heappop(oheap)[1]
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j
            tentative_g_score = gscore[current] + 1
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size:
                if neighbor == obstacle or neighbor in close_set: continue
                if tentative_g_score < gscore.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    gscore[neighbor] = tentative_g_score
                    fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(oheap, (fscore[neighbor], neighbor))
    return []

path_goal_based = a_star_search(GRID_SIZE, START_POS, GOAL_POS, OBSTACLE_POS)

# =============================================================================
# Partie 4 : Comparaison et animation (VERSION CORRIGÉE)
# =============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

def setup_grid(ax, title, path):
    """Prépare une grille STATIQUE pour l'animation."""
    ax.set_title(title)
    ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
    ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)
    ax.tick_params(which="minor", size=0, labelbottom=False, labelleft=False)
    
    # Dessiner les éléments fixes
    ax.add_patch(plt.Rectangle((GOAL_POS[1]-0.5, GOAL_POS[0]-0.5), 1, 1, color='green', alpha=0.7))
    ax.add_patch(plt.Rectangle((OBSTACLE_POS[1]-0.5, OBSTACLE_POS[0]-0.5), 1, 1, color='grey'))
    ax.plot(START_POS[1], START_POS[0], 'ys', markersize=12) # 'ys' = yellow square
    
    # BONUS : Dessiner le chemin complet en pointillé
    if path:
        path_cols = [p[1] for p in path] # x-values
        path_rows = [p[0] for p in path] # y-values
        ax.plot(path_cols, path_rows, color='gray', linestyle='--', linewidth=2, alpha=0.5)


# Initialisation des marqueurs des agents
agent_reflex_marker, = ax1.plot([], [], 'bo', markersize=15, label='Agent Réactif')
agent_goal_based_marker, = ax2.plot([], [], 'ro', markersize=15, label='Agent Intelligent')

def init():
    """Fonction d'initialisation : dessine le fond de chaque animation."""
    setup_grid(ax1, 'Agent Stupide (Réactif)', path_reflex)
    setup_grid(ax2, 'Agent Intelligent (Basé sur un but)', path_goal_based)
    # On initialise la position des agents à vide
    agent_reflex_marker.set_data([], [])
    agent_goal_based_marker.set_data([], [])
    return agent_reflex_marker, agent_goal_based_marker

def animate(i):
    """Met à jour la position des agents à chaque frame."""
    # Mettre à jour l'agent réactif (bleu)
    if i < len(path_reflex):
        row, col = path_reflex[i]
        # set_data attend (x, y), soit (colonne, ligne)
        agent_reflex_marker.set_data([col], [row])
    
    # Mettre à jour l'agent intelligent (rouge)
    if i < len(path_goal_based):
        row, col = path_goal_based[i]
        agent_goal_based_marker.set_data([col], [row])
        
    return agent_reflex_marker, agent_goal_based_marker

# Le nombre de frames est le maximum des longueurs des deux chemins
num_frames = max(len(path_reflex), len(path_goal_based))

ani = FuncAnimation(
    fig, 
    animate, 
    frames=num_frames, 
    init_func=init, 
    blit=True,  # blit=True est plus efficace, il ne redessine que ce qui a changé
    interval=300,
    repeat=True
)

plt.close(fig)
HTML(ani.to_jshtml())

In [6]:
# =============================================================================
# Importation des bibliothèques nécessaires
# =============================================================================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import heapq
import math # On importe la bibliothèque math pour la racine carrée (sqrt)

# =============================================================================
# Partie 1 : Configuration de l'environnement
# =============================================================================
GRID_SIZE = 6
START_POS = (0, 0)  # Format (ligne, colonne)
GOAL_POS = (5, 5)
OBSTACLE_POS = (3, 3)

# =============================================================================
# Partie 2 : Implémentation de l'agent "stupide" (Réactif)
# =============================================================================
# (Aucun changement ici, le comportement reste le même)
path_reflex = [START_POS]
current_pos_reflex = list(START_POS)
for _ in range(GRID_SIZE - 1):
    next_pos = (current_pos_reflex[0], current_pos_reflex[1] + 1)
    if next_pos == OBSTACLE_POS:
        break
    current_pos_reflex[1] += 1
    path_reflex.append(tuple(current_pos_reflex))

# =============================================================================
# Partie 3 : Implémentation de l'agent intelligent (AMÉLIORÉ AVEC DIAGONALES)
# =============================================================================
def heuristic(a, b):
    # L'heuristique de Manhattan reste une bonne estimation (admissible)
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search_diagonal(grid_size, start, goal, obstacle):
    # CHANGEMENT 1 : On autorise maintenant 8 directions de mouvement
    neighbors = [
        (0, 1), (0, -1), (1, 0), (-1, 0),  # Mouvements droits
        (1, 1), (1, -1), (-1, 1), (-1, -1) # Mouvements en diagonale
    ]
    
    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}
    oheap = []
    heapq.heappush(oheap, (fscore[start], start))

    while oheap:
        current = heapq.heappop(oheap)[1]
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j
            
            # CHANGEMENT 2 : Le coût du mouvement dépend de la direction
            # Coût de 1 pour un mouvement droit, sqrt(2) pour une diagonale.
            move_cost = 1 if i == 0 or j == 0 else math.sqrt(2)
            
            tentative_g_score = gscore[current] + move_cost
            
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size:
                if neighbor == obstacle or neighbor in close_set:
                    continue
                if tentative_g_score < gscore.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    gscore[neighbor] = tentative_g_score
                    fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(oheap, (fscore[neighbor], neighbor))
    return []

# On appelle la nouvelle fonction pour calculer le chemin
path_goal_based = a_star_search_diagonal(GRID_SIZE, START_POS, GOAL_POS, OBSTACLE_POS)

# =============================================================================
# Partie 4 : Animation (Aucun changement ici)
# =============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

def setup_grid(ax, title, path):
    ax.set_title(title)
    ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
    ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)
    ax.tick_params(which="minor", size=0, labelbottom=False, labelleft=False)
    ax.add_patch(plt.Rectangle((GOAL_POS[1]-0.5, GOAL_POS[0]-0.5), 1, 1, color='green', alpha=0.7))
    ax.add_patch(plt.Rectangle((OBSTACLE_POS[1]-0.5, OBSTACLE_POS[0]-0.5), 1, 1, color='grey'))
    ax.plot(START_POS[1], START_POS[0], 'ys', markersize=12)
    if path:
        path_cols = [p[1] for p in path]
        path_rows = [p[0] for p in path]
        ax.plot(path_cols, path_rows, color='gray', linestyle='--', linewidth=2, alpha=0.5)

agent_reflex_marker, = ax1.plot([], [], 'bo', markersize=15, label='Agent Réactif')
agent_goal_based_marker, = ax2.plot([], [], 'ro', markersize=15, label='Agent Intelligent')

def init():
    setup_grid(ax1, 'Agent Stupide (Réactif)', path_reflex)
    setup_grid(ax2, 'Agent Intelligent (Optimal)', path_goal_based)
    agent_reflex_marker.set_data([], [])
    agent_goal_based_marker.set_data([], [])
    return agent_reflex_marker, agent_goal_based_marker

def animate(i):
    if i < len(path_reflex):
        row, col = path_reflex[i]
        agent_reflex_marker.set_data([col], [row])
    if i < len(path_goal_based):
        row, col = path_goal_based[i]
        agent_goal_based_marker.set_data([col], [row])
    return agent_reflex_marker, agent_goal_based_marker

num_frames = max(len(path_reflex), len(path_goal_based))
ani = FuncAnimation(
    fig, animate, frames=num_frames, init_func=init, blit=True, interval=300, repeat=True
)

plt.close(fig)
HTML(ani.to_jshtml())